In [7]:
import pandas as pd
import pulp
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, value

# Load the CSV
df = pd.read_csv("bakery_products_5000.csv")

# Group by unique products and take average values (simulate consistent product types)
grouped_df = df.groupby("Product").mean().reset_index()

# Set resource limits (you can tune these values)
resource_limits = {
    "Flour(g)": 100000,
    "Sugar(g)": 60000,
    "Eggs(units)": 1200,
    "Butter(g)": 30000
}

# Initialize the model
model = LpProblem("Bakery_Product_Mix_Optimization", LpMaximize)

# Create decision variables for each unique product
product_vars = {
    row["Product"]: LpVariable(name=row["Product"], lowBound=0, cat="Integer")
    for _, row in grouped_df.iterrows()
}

# Add the objective function: Maximize profit
model += pulp.lpSum(product_vars[row["Product"]] * row["Profit"] for _, row in grouped_df.iterrows()), "Total_Profit"

# Add resource constraints
for resource in resource_limits:
    model += (
        pulp.lpSum(product_vars[row["Product"]] * row[resource] for _, row in grouped_df.iterrows()) 
        <= resource_limits[resource], 
        f"{resource}_Constraint"
    )

# Solve the problem
model.solve()

# Print the status
print(f"📌 Status: {LpStatus[model.status]}")

# Print optimal quantities for each product
print("\n✅ Optimal Production Plan:")
for product in product_vars:
    qty = product_vars[product].varValue
    if qty > 0:
        print(f"  {product}: {qty} units")

# Print max profit
print(f"\n💰 Maximum Profit: ${value(model.objective)}")

# Resource usage
print("\n📦 Resource Utilization:")
for resource in resource_limits:
    used = sum(product_vars[row["Product"]].varValue * row[resource] for _, row in grouped_df.iterrows())
    print(f"  {resource}: {used:.2f}/{resource_limits[resource]} used")


📌 Status: Optimal

✅ Optimal Production Plan:
  Bread: 139.0 units
  Donut: 31.0 units
  Muffin: 627.0 units

💰 Maximum Profit: $26277.713906635985

📦 Resource Utilization:
  Flour(g): 99393.36/100000 used
  Sugar(g): 59980.66/60000 used
  Eggs(units): 797.00/1200 used
  Butter(g): 29998.98/30000 used


In [1]:
pip install pulp


   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
    --------------------------------------- 0.3/16.4 MB ? eta -:--:--
   -- ------------------------------------- 1.0/16.4 MB 3.0 MB/s eta 0:00:06
   ---- ----------------------------------- 1.8/16.4 MB 3.5 MB/s eta 0:00:05
   ------- -------------------------------- 3.1/16.4 MB 4.0 MB/s eta 0:00:04
   ---------- ----------------------------- 4.5/16.4 MB 4.5 MB/s eta 0:00:03
   ------------ --------------------------- 5.2/16.4 MB 4.7 MB/s eta 0:00:03
   --------------- ------------------------ 6.3/16.4 MB 4.4 MB/s eta 0:00:03
   ----------------- ---------------------- 7.1/16.4 MB 4.5 MB/s eta 0:00:03
   ------------------- -------------------- 8.1/16.4 MB 4.4 MB/s eta 0:00:02
   --------------------- ------------------ 8.9/16.4 MB 4.4 MB/s eta 0:00:02
   ------------------------ --------------- 10.2/16.4 MB 4.5 MB/s eta 0:00:02
   ---------------------------- ----------- 11.5/16.4 MB 4.6 MB/s eta 0:00:02
   -------